In [ ]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)

    except RuntimeError as e:
        print(e)

In [ ]:
import tensorflow as tf
tf.test.is_gpu_available()
tf.test.is_built_with_cuda()

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
!nvidia-smi

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
import os
import matplotlib.pyplot as plt
import cv2
import numpy as np
import random
import time
import argparse
from keras.preprocessing import image
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import Conv2D,BatchNormalization, LeakyReLU, ZeroPadding2D,MaxPooling2D
import keras
from keras.models import Model
from keras.layers import Conv2D, MaxPool2D,  \
    Dropout, Dense, Input, concatenate,      \
    GlobalAveragePooling2D, AveragePooling2D,\
    Flatten
from keras import backend as K 
from keras.models import Sequential
from keras.utils import np_utils
import math 
from keras.optimizers import SGD, Adam
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, LearningRateScheduler
from keras.regularizers import l1 , l2
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, roc_auc_score, classification_report, roc_curve
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from imutils import paths

In [ ]:
covid_path = list(paths.list_images('...'))
data1 = []
labels1 = []
for imagePath in covid_path:
    label = 'Covid'
    # load the image, swap color channels, and resize it to be a fixed
    # 224x224 pixels while ignoring aspect ratio
    image = cv2.imread(imagePath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))
    # update the data and labels lists, respectively
    data1.append(image)
    labels1.append(label)
print('Covid:', len(labels1))

normal_path = list(paths.list_images('...'))
data2 = []
labels2 = []
for imagePath in normal_path:
    label = 'Normal'
    # load the image, swap color channels, and resize it to be a fixed
    # 224x224 pixels while ignoring aspect ratio
    image = cv2.imread(imagePath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))
    # update the data and labels lists, respectively
    data2.append(image)
    labels2.append(label)
print('Normal:', len(labels2))

viral_path = list(paths.list_images('...'))
data3 = []
labels3 = []
for imagePath in viral_path:
    label = 'Viral Pneumonia'
    # load the image, swap color channels, and resize it to be a fixed
    # 224x224 pixels while ignoring aspect ratio
    image = cv2.imread(imagePath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))
    # update the data and labels lists, respectively
    data3.append(image)
    labels3.append(label)
print('Viral Pneumonia:', len(labels3))


bacteria_path = list(paths.list_images('...'))
data4 = []
labels4 = []
for imagePath in bacteria_path:
    label = 'Bacterial Pneumonia'
    # load the image, swap color channels, and resize it to be a fixed
    # 224x224 pixels while ignoring aspect ratio
    image = cv2.imread(imagePath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))
    # update the data and labels lists, respectively
    data4.append(image)
    labels4.append(label)
print('Bacterial Pneumonia:', len(labels4))


lung_opacity_path = list(paths.list_images('...'))
data5 = []
labels5 = []
for imagePath in lung_opacity_path:
    label = 'Lung Opacity'
    # load the image, swap color channels, and resize it to be a fixed
    # 224x224 pixels while ignoring aspect ratio
    image = cv2.imread(imagePath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))
    # update the data and labels lists, respectively
    data5.append(image)
    labels5.append(label)
print('Lung Opacity:', len(labels5))

In [ ]:
import random
def shuffle_data(x, y):
  tmp = list(zip(x, y)) 
  random.shuffle(tmp)
  x, y = zip(*tmp)
  x, y = list(x), list(y)
  del(tmp)
  return x, y
data1, labels1 = shuffle_data(data1, labels1)
data2, labels2 = shuffle_data(data2, labels2)
data3, labels3 = shuffle_data(data3, labels3)
data4, labels4 = shuffle_data(data4, labels4)
data5, labels5 = shuffle_data(data5, labels5)

total_data = data1 + data3 + data2 + data5 + data4
total_labels = labels1 + labels3 + labels2 + labels5 + labels4
print('Total data:', len(total_data))
print('Total labels:', len(total_labels))
total_data, total_labels = shuffle_data(total_data, total_labels)

In [ ]:
data = np.array(total_data) / 255.0
label = np.array(total_labels)
# encode class values as integers
lb = LabelEncoder()
lb.fit(label)
print(label[18])
label = lb.transform(label)
# convert integers to dummy variables (i.e. one hot encoded)
label = np_utils.to_categorical(label)
print(label[18])

In [ ]:
import numpy as np


def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):

    import matplotlib.pyplot as plt
    import numpy as np
    import itertools

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.grid(False)
    plt.ylabel('True label')
    plt.xlabel('Predicted label'.format(accuracy, misclass))
    plt.show()

In [ ]:
#my functions
def conv_block (x,nf,kernel_size):
  #conv layer with BN and LeakyReLU
  x = Conv2D(nf, kernel_size, padding='same', strides=(1,1), activation=None)(x)
  x = BatchNormalization(trainable=True)(x)
  x = LeakyReLU(alpha=0.1)(x)
  return x


def maxpooling(x):
  x= MaxPooling2D((2,2), strides=(2,2))(x)
  return x

def fire_conv(x,nf):
    x = conv_block (x,nf//2,(3,3))
    x = maxpooling(x)
    x = conv_block (x,nf//4,(1,1))
    x1 = ZeroPadding2D(padding=(1, 1))(x)
    x1 = conv_block (x1,nf,(3,3))
    x2 = ZeroPadding2D(padding=(1, 1))(x)
    x2 = conv_block (x2,nf,(1,1))
    x = keras.layers.concatenate([x1, x2], axis = 3)
    return x

In [ ]:
#FireCovNet

def create_model(reg=0.0001, opt='Adam'):
    inputs = Input(shape=(224, 224, 3))
    x = Sequential()
    x = conv_block(inputs, 8,(3,3))
    x = maxpooling(x)
    x = fire_conv(x, 32)
    x = fire_conv(x, 64)
    x = fire_conv(x, 128)
    x = fire_conv(x, 256)
    x = Conv2D(5, (1,1), padding='same', strides=(1,1), activation='relu',kernel_regularizer=l2(reg), bias_regularizer=l2(reg))(x)
    x = BatchNormalization(trainable=True)(x)
    x = GlobalAveragePooling2D()(x)
    x = Dense(5, activation='softmax', name='predictions')(x)
    model = Model(inputs, x)
    for layer in model.layers:
        layer.trainable = True
    model.compile(loss='categorical_crossentropy', optimizer=opt , metrics=['accuracy'])
    model.summary()
    return model

In [ ]:
checkpoint = ModelCheckpoint(filepath='my-model-{val_accuracy:.2f}.hdf5',monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=True)
lr_callback = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=3, verbose=1, mode='auto')

In [ ]:
# Initialize W&B run for experiment tracking
import wandb
from wandb.keras import WandbCallback
wandb.login()

In [ ]:
#Training FirecovNet using 5fold cross validation

fold_no = 1
batch_size = 8
epochs = 50
class_no=5

acc_per_fold = []
loss_per_fold = []
cm_per_fold = []

# Define the K-fold Cross Validator
kfold = KFold(n_splits=5, shuffle=True)

for train, test in kfold.split(data, label):
    keras.backend.clear_session()
    model = create_model()
    wandb.init(project="firecovnet", entity="hassanlou",job_type='train', name=f'fold_{fold_no}_mine(bclnv)_{class_no}')
    callbacks = [WandbCallback(), lr_callback, checkpoint]
    history = model.fit(data[train],label[train], batch_size=8, epochs=50, verbose=1, validation_split=0.1 , callbacks=callbacks)
    model.save(f'my_model_fold_{fold_no}.h5')
    # Evaluate the model with the metrics we defined earlier  
    print("[INFO] evaluating network...")
    (loss, accuracy) = model.evaluate(data[test], label[test], batch_size=8, verbose=1)
    print('Test loss:', loss)
    print('Test accuracy:', accuracy*100)
    wandb.log({'Test loss': loss, 'Test accuracy': accuracy})
    acc_per_fold.append(accuracy* 100)
    loss_per_fold.append(loss)
    print(len(label[test]))
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {loss}; {model.metrics_names[1]} of {accuracy*100}%')
  
    predIdxs = model.predict(data[test], batch_size=8)
    predIdxs = np.argmax(predIdxs, axis=1)
    print(classification_report(label[test].argmax(axis=1), predIdxs, target_names=lb.classes_))
    cm = confusion_matrix(label[test].argmax(axis=1), predIdxs)
    wandb.log({'confusion matrix': cm})
    print(cm)
    cm_per_fold.append(cm)
    wandb.join()
    fold_no += 1
    

# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
  keras.backend.clear_session()
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
cm_total= cm_per_fold[0]+cm_per_fold[1]+cm_per_fold[2]+cm_per_fold[3]+cm_per_fold[4]

print('------------------------------------------------------------------------')


plot_confusion_matrix(cm           = cm_total, 
                      normalize    = False,
                      target_names = ['Bacterial Pneumonia','Covid','Lung Opacity', 'Normal', 'Viral Pneumonia'],
                      title        = "Overlapped Confusion Matrix")
plot_confusion_matrix(cm           = cm_total, 
                      normalize    = True,
                      target_names = ['Bacterial Pneumonia','Covid','Lung Opacity', 'Normal', 'Viral Pneumonia'],
                      title        = "Overlapped Confusion Matrix")

In [ ]:

snn_pred = model.predict(data[test], batch_size=8, verbose=1) 
snn_predicted = np.argmax(snn_pred, axis=1)

In [ ]:
from sklearn.datasets import make_classification
from sklearn.preprocessing import label_binarize
from scipy import interp
from itertools import cycle

n_classes = 5

from sklearn.metrics import roc_curve, auc

# Plot linewidth.
lw = 2
plt.figure(figsize = (8,6))
#plt.figure(2)
plt.xlim(-0.008, 0.2)
plt.ylim(0.7, 1.005)
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(label[test][:, i], snn_pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(label[test].ravel(), snn_pred.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
# Compute macro-average ROC curve and ROC area

# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])
# plotting   
# 
plt.plot(fpr["micro"], tpr["micro"], label='micro-average ROC curve (area = {0:0.2f})'
''.format(roc_auc["micro"]),color='deeppink', linestyle=':', linewidth=2)

plt.plot(fpr["macro"], tpr["macro"], label='macro-average ROC curve (area = {0:0.2f})'
''.format(roc_auc["macro"]),color='navy', linestyle=':', linewidth=2) 
plt.plot(fpr[0], tpr[0], linestyle='-',color='orange', label='ROC curve of Bacterial Pneumonia (area = {1:0.2f})'
''.format(0, roc_auc[0]))
plt.plot(fpr[1], tpr[1], linestyle='-',color='green', label='ROC curve of Covid-19 (area = {1:0.2f})'
''.format(1, roc_auc[1]))
plt.plot(fpr[2], tpr[2], linestyle='-',color='yellow', label='ROC curve of Lung Opacity (area = {1:0.2f})'
''.format(2, roc_auc[2]))
plt.plot(fpr[3], tpr[3], linestyle='-',color='blue', label='ROC curve of Normal (area = {1:0.2f})'
''.format(3, roc_auc[3]))
plt.plot(fpr[4], tpr[4], linestyle='-',color='red', label='ROC curve of Viral Pneumonia (area = {1:0.2f})'
''.format(4, roc_auc[4]))


plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.grid(True)
plt.title('ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive rate')
plt.legend(loc='best')
plt.savefig('Multiclass ROC',dpi=300);  

In [ ]:
from sklearn.datasets import make_classification
from sklearn.preprocessing import label_binarize
from scipy import interp
from itertools import cycle

n_classes = 5

from sklearn.metrics import roc_curve, auc

# Plot linewidth.
lw = 2
plt.figure(figsize = (8,6))
#plt.figure(2)
plt.xlim(-0.02, 1.02)
plt.ylim(0, 1.02)
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(label[test][:, i], snn_pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(label[test].ravel(), snn_pred.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
# Compute macro-average ROC curve and ROC area

# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])
# plotting   
# 
plt.plot(fpr["micro"], tpr["micro"], label='micro-average ROC curve (area = {0:0.2f})'
''.format(roc_auc["micro"]),color='deeppink', linestyle=':', linewidth=2)

plt.plot(fpr["macro"], tpr["macro"], label='macro-average ROC curve (area = {0:0.2f})'
''.format(roc_auc["macro"]),color='navy', linestyle=':', linewidth=2) 
plt.plot(fpr[0], tpr[0], linestyle='-',color='orange', label='ROC curve of Bacterial Pneumonia (area = {1:0.2f})'
''.format(0, roc_auc[0]))
plt.plot(fpr[1], tpr[1], linestyle='-',color='green', label='ROC curve of Covid-19 (area = {1:0.2f})'
''.format(1, roc_auc[1]))
plt.plot(fpr[2], tpr[2], linestyle='-',color='yellow', label='ROC curve of Lung Opacity (area = {1:0.2f})'
''.format(2, roc_auc[2]))
plt.plot(fpr[3], tpr[3], linestyle='-',color='blue', label='ROC curve of Normal (area = {1:0.2f})'
''.format(3, roc_auc[3]))
plt.plot(fpr[4], tpr[4], linestyle='-',color='red', label='ROC curve of Viral Pneumonia (area = {1:0.2f})'
''.format(4, roc_auc[4]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.grid(True)
plt.title('ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive rate')
plt.legend(loc='best')
plt.savefig('Multiclass ROC',dpi=300);  

In [ ]:
def img_to_array(img, data_format='channels_last', dtype='float32'):
    """Converts a PIL Image instance to a Numpy array.
    # Arguments
        img: PIL Image instance.
        data_format: Image data format,
            either "channels_first" or "channels_last".
        dtype: Dtype to use for the returned array.
    # Returns
        A 3D Numpy array.
    # Raises
        ValueError: if invalid `img` or `data_format` is passed.
    """
    if data_format not in {'channels_first', 'channels_last'}:
        raise ValueError('Unknown data_format: %s' % data_format)
    # Numpy array x has format (height, width, channel)
    # or (channel, height, width)
    # but original PIL image has format (width, height, channel)
    x = np.asarray(img, dtype=dtype)
    if len(x.shape) == 3:
        if data_format == 'channels_first':
            x = x.transpose(2, 0, 1)
    elif len(x.shape) == 2:
        if data_format == 'channels_first':
            x = x.reshape((1, x.shape[0], x.shape[1]))
        else:
            x = x.reshape((x.shape[0], x.shape[1], 1))
    else:
        raise ValueError('Unsupported image shape: %s' % (x.shape,))
    return x